# 站間量分析

需要準備資料：
1. 票證資料：須包含所有佔位點的資料
2. 站序資料：需帶有'Direction'欄位
3. 班表資料：需帶有'Direction'、'IsWorkday'欄位
4. 營運月報 (optional)：做票證放大率佐證用 

## 基礎設定

包含環境設定，以及指定對應資料夾路徑（input、process、output）

In [13]:
import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from tickets_cleaning import tickets_cleaning, date_defined , getDaysCount, getMagnification, tickets_match_shift

In [14]:
inputfolder_path = os.path.join(os.getcwd(),'..', 'input')
outputfolder_path = os.path.join(os.getcwd(),'..', 'output')
processfolder_path = os.path.join(os.getcwd(),'..', 'process')

# 確保資料夾存在
os.makedirs(inputfolder_path, exist_ok=True)
os.makedirs(outputfolder_path, exist_ok=True)
os.makedirs(processfolder_path, exist_ok=True)


In [15]:
# 加入補班、國定假日等日期  Note:需要手動調整
date_turn_holiday=[20230929] # 補假、國定假日、颱風天
date_turn_workday=[20230923] # 補班
startdate = 20230701
enddate = 20230930
# 計算每月的假日與平日數
dayscount = getDaysCount(startdate, enddate, date_turn_holiday, date_turn_workday)


In [16]:
# 定義票證資料的欄位名稱 Note:需要手動調整
direction_col = 'DIRECTION'
getontime_col ='GETON_DATE'
getofftime_col ='GETOFF_DATE'
getonstop_col ='GETON_STOP_NAME'
getoffstop_col ='GETOFF_STOP_NAME'
getonseq_col ='GETON_STOP_SEQ'
getoffseq_col ='GETOFF_STOP_SEQ'
routename_col = "ROUTE_NAME"

## 資料前處理

1. 票證清洗(去除不可用資料)
2. 票證定義日期欄位 (年月、平假日)
3. 處理票證放大率

In [17]:
'''進行基礎的票證清洗
1. 找到上車時間 < 下車時間
2. 上車站序 < 下車站序
3. 上下車站名不同'''

# 定義 tickets.csv 的相對路徑
tickets_path = os.path.join(inputfolder_path , 'tickets.csv')
tickets = pd.read_csv(tickets_path)

# 讀取資料並進行清理
tickets, errorstat, correctrate = tickets_cleaning(tickets, 
    getontime=getontime_col, 
    getofftime=getofftime_col, 
    getonstop=getonstop_col, 
    getoffstop=getoffstop_col, 
    getonseq=getonseq_col, 
    getoffseq=getoffseq_col)

# 把清洗過的資料轉存至process
tickets.to_csv(os.path.join(processfolder_path , 'tickets_cleaned.csv'))
# tickets.to_csv(os.path.join(os.path.dirname(__file__), '..', 'process', 'tickets_cleaned.csv'))

# 輸出數據清洗統計
errorstat_path = os.path.join(outputfolder_path , 'ErrorDataStat.txt')
with open(errorstat_path , 'w', encoding='utf-8') as file:
    for key, value in errorstat.items():
        file.write(f"{key}: {value}\n")
# del errorstat
tickets = date_defined(tickets, getontime_columns=getontime_col, date_turn_holiday=date_turn_holiday,\
                       date_turn_workday=date_turn_workday)


C:\Users\corgi\AppData\Local\Temp\ipykernel_22304\3120557935.py:8: DtypeWarning: Columns (4,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets = pd.read_csv(tickets_path)


In [ ]:
# tickets = pd.read_csv(os.path.join(processfolder_path , 'tickets_cleaned.csv'))

In [18]:
'''處理票證資料放大率'''

operation = pd.read_csv(os.path.join(inputfolder_path, 'operation.csv'))

# 計算 DataYearMonth 並格式化
operation['DataYearMonth'] = (
    pd.to_datetime((operation['YEAR'] + 1911) * 100 + operation['MONTH'], format='%Y%m')
    .dt.strftime('%Y%m')
)

tickets_magnification = getMagnification(
    tickets=tickets,
    tickets_routename_col=routename_col,
    tickets_yearmonth_col='DataYearMonth',  # 指定票證數據的年月欄位 Note:需要手動調整
    operation=operation,
    operation_routename_col='ROUTE_NAME', # 這個Operation須要手動調整欄位名稱 Note:需要手動調整
    operation_yearmonth_col='DataYearMonth',  # 指定運營數據的年月欄位 Note:需要手動調整
    operation_passengers_col='PASSENGERS' # Note:需要手動調整
)

# 列出所有放大率會有異常的路線
ooc_route_list = list(set(tickets_magnification[tickets_magnification['Magnification'] >= 1.3]['RouteName'].unique()).union(
    set(tickets_magnification[tickets_magnification['Magnification'] <= 0.8]['RouteName'].unique())
))

In [ ]:
operation.columns

In [38]:
def operation_calcuate(df, datayearmonth_col='DataYearMonth',
                        operator_col='Operator', routename_col='RouteName', 
                        drivingmiles_col='DrivingMiles', shift_col='Shifts',
                        passengers_col='Passengers', passengerkilometers_col='PassengerKilometers',
                        income_col='Income', dayscountdf = dayscount):
    """
    修正錯誤並優化函式的資料處理邏輯。

    Parameters:
    - df: pandas.DataFrame
    - datayearmonth_col: str, 預設為 'DataYearMonth'
    - operator_col: str, 預設為 'Operator'
    - routename_col: str, 預設為 'RouteName'
    - drivingmiles_col: str, 預設為 'DrivingMiles'
    - shift_col: str, 預設為 'Shifts'
    - passengers_col: str, 預設為 'Passengers'
    - passengerkilometers_col: str, 預設為 'PassengerKilometers'
    - income_col: str, 預設為 'Income'
    - dayscount: pandas.DataFrame 為之前算出來的天數 

    Returns:
    - pandas.DataFrame, 修正後的資料框
    """
    # 填補 NaN
    df = df.fillna(0)
    
    # 移除重複值
    df = df.drop_duplicates()

    # 將資料轉型
    df[[operator_col, routename_col]] = df[[operator_col, routename_col]].astype(str)
    df[[drivingmiles_col, shift_col, passengers_col, passengerkilometers_col, income_col]] = df[[drivingmiles_col, shift_col, passengers_col, passengerkilometers_col, income_col]].astype(float)

    # 營運指標計算
    dayscountdf['Days'] = dayscountdf['Holiday'] + dayscountdf['Workday']
    df = pd.merge(df, dayscountdf[['DataYearMonth','Days']], left_on=datayearmonth_col, right_on='DataYearMonth')
    df['DailyShifts'] = df[shift_col] / df['Days'] #  每日平均班次
    df['PassengersPerShift'] = df[passengers_col] / df[shift_col] # 每班次平均載客數
    df['PassengersPerDay'] = df[passengers_col] / df['Days'] # 每日平均載客
    df['KilometersPerPassengers'] = df[passengerkilometers_col] / df[passengers_col] # 每人平均搭乘里程
    df['PassengersPerKilometers'] = df[passengers_col] / df[drivingmiles_col]  # 每公里載客
    df['IncomePerKilometers'] = df[income_col] / df[passengerkilometers_col] # 每公里營收
    return df


In [ ]:
def define_quadrant(df, groupbycolumns, xcol, ycol, measure='mean'):
    """
    定義四象限，根據 xcol 和 ycol 的值進行比較來找出重點。

    Parameters:
        df (pd.DataFrame): 原始資料框。
        groupbycolumns (list): 需要進行分組的欄位。
        xcol (str): x象限上的欄位。
        ycol (str): y象限上的欄位。
        measure (str): 'mean'、'median' 或 'mode'，用來計算 x 和 y 軸的值，預設為 'mean'。

    Returns:
        pd.DataFrame: 加入象限分類的資料框。
    """
    # 確保傳入的 measure 是有效的
    if measure not in ['mean', 'median', 'mode']:
        raise ValueError("Invalid measure. Choose from 'mean', 'median', or 'mode'.")

    # 複製原始資料框
    df_how = df.copy()

    # 定義 measure 映射
    measure_map = {
        'mean': lambda x: x.mean(),
        'median': lambda x: x.median(),
        'mode': lambda x: x.mode().iloc[0] if not x.mode().empty else None
    }

    # 使用 groupby 和 agg 來計算對應的統計數值
    df_how = df.groupby(groupbycolumns).agg({
        xcol: measure_map[measure],
        ycol: measure_map[measure]
    }).rename(columns={xcol: 'x', ycol: 'y'}).reset_index()

    # 合併原始數據框和計算過的數據框
    df = pd.merge(df, df_how, on=groupbycolumns, how='left')

    # 根據 xcol 和 ycol 的值與 x 和 y 比較，為資料點分配四象限
    def assign_quadrant(row):
        if row[xcol] > row['x'] and row[ycol] > row['y']:
            return 'Q1'  # 第一象限
        elif row[xcol] < row['x'] and row[ycol] > row['y']:
            return 'Q2'  # 第二象限
        elif row[xcol] < row['x'] and row[ycol] < row['y']:
            return 'Q3'  # 第三象限
        elif row[xcol] > row['x'] and row[ycol] < row['y']:
            return 'Q4'  # 第四象限
        else:
            return 'On the Border'  # 如果正好在邊界上，歸類為邊界區域

    # 為資料框中的每一列分配象限
    df['quadrant'] = df.apply(assign_quadrant, axis=1)

    return df


In [44]:
operationcount = operation_calcuate(df = operation , operator_col='OPERATOR_NAME', routename_col='ROUTE_NAME',
                        drivingmiles_col='OPERATING_DRIVING_MILES', shift_col='VEHICLE_SHIFTS',
                        passengers_col='PASSENGERS', passengerkilometers_col='PASSENGER_KILOMETERS',
                        income_col='INCOME', dayscountdf = dayscount)
operationcount = define_quadrant(operationcount, ['DataYearMonth'], xcol='DailyShifts', ycol='PassengersPerKilometers', measure='mean')
operationcount.to_csv(os.path.join(outputfolder_path, 'operation_yearmonth.csv'), index = False, encoding= 'big5')

In [ ]:
# # 讀取班表資料
# shift = pd.read_excel(os.path.join(inputfolder_path, 'shift.xlsx'))
# shift.columns = ['RouteName', 'Direction', 'Shift', 'IsWorkday']
# shift['IsWorkday'] = shift['IsWorkday'].replace({'假日': '0', '平日': '1'})
# shift['Shift'] = shift['Shift'].astype(str)
# shift['Shift'] = pd.to_datetime(shift['Shift'], format='%H:%M').dt.time
# shift = shift.sort_values(['RouteName', 'IsWorkday','Shift', 'Direction'], ascending=[True, True, True, True])

# 讀取班表資料
shift = pd.read_csv(os.path.join(inputfolder_path, 'shift.csv'))
shift['Shift'] = shift['Shift'].astype(str)
shift['Shift'] = pd.to_datetime(shift['Shift'], format='%H:%M').dt.time
shift = shift.sort_values(['RouteName', 'IsWorkday','Shift', 'Direction'], ascending=[True, True, True, True]).reset_index(drop = True)

# 讀取相關的站序 
seq = pd.read_csv(os.path.join(inputfolder_path,'seq.csv'))

# 具有班表的RouteName_list
shift_routename_list = list(shift['RouteName'].unique()) 
tickets_routename_list = list(tickets[routename_col].unique())
seq_routename_list = list(shift['RouteName'].unique())

# 不在 tickets_routename_list 中但在 shift_routename_list 中的項目
only_in_shift = list(set(shift_routename_list) - set(tickets_routename_list))
# 不在 shift_routename_list 中但在 tickets_routename_list 中的項目
only_in_tickets = list(set(tickets_routename_list) - set(shift_routename_list))
common_routes = list(set(tickets_routename_list) & set(shift_routename_list) & set(seq_routename_list))

# 印出結果
print("缺票證資料:", only_in_shift)
print("缺班表資料:", only_in_tickets)
print("本次可算的路線:", common_routes)

### 基本判讀指標：是否繼續往下做

1. 列出本次資料正常資料的佔比
2. 列出本次放大率異常的路線 ( 可以進一步以plotly 圖表檢視長條圖)

In [ ]:
print(f'資料可用比例 = {correctrate}%',end=' ')
if correctrate <= 95:
    print('本次取得的資料錯誤率太高，建議重新檢視')
else : 
    print('本次的資料可以使用')

try:
    if len(ooc_route_list) > 0:
        print(f'本次放大率異常路線共{len(ooc_route_list)}條')
        print('票證放大率異常的路線編號', end= ':')
        print(ooc_route_list)
except:
    pass

In [ ]:
unique_year_months = tickets_magnification["DataYearMonth"].unique()
# 創建篩選器 (Dropdown)
dropdown = widgets.Dropdown(
    options=unique_year_months,
    value=unique_year_months[0],
    description="月份:"
)

# 定義繪圖函數
def plot_barchart(selected_month):
    # 篩選 DataFrame
    filtered_df = tickets_magnification[tickets_magnification["DataYearMonth"] == selected_month]
    
    if filtered_df.empty:
        print(f"No data available for {selected_month}")
        return
    
    # 創建條形圖
    fig = go.Figure()

    # 定義顯示在 hover 上的格式
    hover_text_tickets = [
    f"RouteName: {row['RouteName']}<br>Magnification: {row['Magnification'] * 100:.2f}%<br>Tickets: {row['Tickets']:,}"  # Magnification 顯示為百分比，Tickets 顯示為實際數字
    for _, row in filtered_df.iterrows()
    ]
    hover_text_passengers = [
    f"RouteName: {row['RouteName']}<br>Magnification: {row['Magnification'] * 100:.2f}%<br>Passengers: {row['Passengers']:,}"  # Magnification 顯示為百分比，Passengers 顯示為實際數字
    for _, row in filtered_df.iterrows()
    ]

    # 添加 Tickets 的長條圖
    fig.add_trace(go.Bar(
        x=filtered_df["RouteName"],
        y=filtered_df["Tickets"],
        name="Tickets",
        marker_color="#84C1FF",
        hovertext=hover_text_tickets,  # 顯示格式化過的 hovertext
        hoverinfo="text"  # 只顯示 hovertext 的內容
    ))

    # 添加 Passengers 的長條圖
    fig.add_trace(go.Bar(
        x=filtered_df["RouteName"],
        y=filtered_df["Passengers"],  # 更新欄位名稱為 Passengers
        name="Passengers",
        marker_color="#FF8000",
        hovertext=hover_text_passengers,  # 顯示格式化過的 hovertext
        hoverinfo="text"  # 只顯示 hovertext 的內容
    ))

    # 設定標題與軸標籤
    fig.update_layout(
        title=f"Tickets and Passengers for {selected_month}",
        xaxis_title="路線編號",
        yaxis_title="人次",
        barmode="group",  # 並列顯示長條圖
        xaxis_tickangle=-90,
        template="plotly_white"  # 使用白色背景的模板
    )

    # 顯示圖表
    fig.show()

# 綁定事件到篩選器
dropdown.observe(lambda change: plot_barchart(change.new), names="value")

# 初始顯示
display(dropdown)
plot_barchart(dropdown.value)


## 資料運算

逐條路線執行
1. 比對班次：依據上車時間比對發車班次（適用班次兼具較少的班次）
2. 班表 - 站序配對
3. 計算**上下車人次**及**站間量**

In [ ]:
outputfolder_unmagnification_path = os.path.join(os.getcwd(),'..', 'output','未放大過的每班次載客')
os.makedirs(outputfolder_unmagnification_path, exist_ok=True)

# addlist = []

# 迴圈計算
for route in common_routes[138:]: # 以前三條路線進行
    print(route, end = " ")
    routefilename = os.path.join(outputfolder_unmagnification_path, f'{route}.xlsx')
    
    with pd.ExcelWriter(routefilename) as writer:
        # 篩選特定路線的資料
        seq_select = seq[seq['RouteName'] == route].reset_index(drop=True)
        shift_select = shift[shift['RouteName'] == route].reset_index(drop=True)
        
        # 篩選符合月份的票證資料
        tickets_select = tickets[tickets[routename_col] == route].sort_values(getontime_col).reset_index(drop=True)
        yearmonthlist = list(tickets['DataYearMonth'].unique())
        
        onandoff_MAX_list = []

        # 針對每個月份進行處理
        for yearmonth in yearmonthlist:
            sheetname = str(yearmonth)
            
            # 呼叫平假日天數
            holidaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Holiday'].values[0]
            Workdaycount = dayscount[dayscount['DataYearMonth'] == yearmonth]['Workday'].values[0]
            shift_select['Days'] = shift_select['IsWorkday'].apply(lambda x: Workdaycount if x == 1 else holidaycount)
            
            # 合併班表和路線序列
            seqwithshift = pd.merge(shift_select, seq_select, on=['RouteName', 'Direction'])

            # 篩選該月份的票證資料
            tickets_select_month = tickets_select[tickets_select['DataYearMonth'] == yearmonth]
            tickets_select_month = tickets_select_month[tickets_select_month[direction_col].isin(list(shift_select['Direction'].unique()))].reset_index(drop=True)
            
            # 比對班表
            tickets_select_month = tickets_match_shift(
                tickets=tickets_select_month,
                shifts=shift_select,
                routename_col=routename_col,
                getontime_col=getontime_col,
                direction_col=direction_col
            )
            
            # 計算每站上下車數據
            onandoff = pd.merge(
                seqwithshift,
                tickets_select_month.groupby([routename_col, direction_col, 'IsWorkday', getonseq_col, 'Matched_Shift'])
                .size().reset_index(name='OnCount')
                .rename(columns={routename_col: 'RouteName', direction_col: 'Direction', getonseq_col: 'Seq', 'Matched_Shift': 'Shift'}),
                on=['RouteName', 'Direction', 'IsWorkday', 'Seq', 'Shift'], how='left'
            )
            onandoff = pd.merge(
                onandoff,
                tickets_select_month.groupby([routename_col, direction_col, 'IsWorkday', getoffseq_col, 'Matched_Shift'])
                .size().reset_index(name='OffCount')
                .rename(columns={routename_col: 'RouteName', direction_col: 'Direction', getoffseq_col: 'Seq', 'Matched_Shift': 'Shift'}),
                on=['RouteName', 'Direction', 'IsWorkday', 'Seq', 'Shift'], how='left'
            )
            onandoff[['OnCount', 'OffCount']] = onandoff[['OnCount', 'OffCount']].fillna(0).astype(int)
            
            # 計算站間累計人數
            for i in range(len(onandoff)):
                if onandoff.loc[i, 'Seq'] == 1:
                    onandoff.loc[i, 'OnBus'] = onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
                else:
                    onandoff.loc[i, 'OnBus'] = onandoff.loc[i - 1, 'OnBus'] + onandoff.loc[i, 'OnCount'] - onandoff.loc[i, 'OffCount']
            onandoff.loc[onandoff['OnBus'] < 0, 'OnBus'] = 0
            
            # 平均上下車數
            onandoff['OnAVG'] = round(onandoff['OnCount'] / onandoff['Days'], 1)
            onandoff['OffAVG'] = round(onandoff['OffCount'] / onandoff['Days'], 1)
            
            # 平均站間人數
            for i in range(len(onandoff)):
                if onandoff.loc[i, 'Seq'] == 1:
                    onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
                else:
                    onandoff.loc[i, 'OnBusAvg'] = onandoff.loc[i - 1, 'OnBusAvg'] + onandoff.loc[i, 'OnAVG'] - onandoff.loc[i, 'OffAVG']
            onandoff.loc[onandoff['OnBusAvg'] < 0, 'OnBusAvg'] = 0
            onandoff['DataYearMonth'] = yearmonth
            onandoff =  onandoff[[ 'DataYearMonth'] + [col for col in  onandoff.columns if col != 'DataYearMonth']]
            addlist.append(onandoff)
            
            # 寫入該月份的數據
            onandoff.to_excel(writer, sheet_name=sheetname, index=False)
            
            # 計算每月最大站間量
            onandoff_MAX = onandoff.groupby(['RouteName', 'Direction']).agg(OnBusMax=('OnBusAvg', 'max')).reset_index()
            onandoff_MAX['DataYearMonth'] = yearmonth
            onandoff_MAX = onandoff_MAX[[ 'DataYearMonth'] + [col for col in onandoff_MAX.columns if col != 'DataYearMonth']]
            onandoff_MAX_list.append(onandoff_MAX)
            
        
        # 整理每月最大站間量並寫入
        if onandoff_MAX_list:
            onandoff_MAX_list = pd.concat(onandoff_MAX_list)
            onandoff_MAX_list.to_excel(writer, sheet_name="每月最大站間量", index=False)
            print('done')


In [ ]:
allstop = pd.concat(addlist)
allstop.to_csv(os.path.join(os.getcwd(),'..', 'output','Busflow.csv'), index=False)

### 轉換為每一個站間的人數多寡

In [ ]:
# 產生OD帶寬
def get_OD_line_shp(df, o_col, d_col, o_x_col, o_y_col, d_x_col, d_y_col, count_col, how = 'countd', date_col ,combine = True , span = 100):
    '''
    Parameters:
    df (dataframe) : 要計算的表格，例如信令資料、票證資料統計圖表
    o_col (str) : 起點的名稱 / 站序 / 可判別的欄位
    d_col (str) : 迄點的名稱 / 站序 / 可判別的欄位
    o_x_col (str) : 起點的經度 (wgs84)
    o_y_col (str) : 起點的緯度 (wgs84)
    d_x_col (str) : 迄點的經度 (wgs84)
    d_y_col (str) : 迄點的緯度 (wgs84)
    count_col (str) : 需要統計依據的量值
    combine (Boolean) : True為去定義是否要把同一對起訖端點的量合併成同一條統計量，False則僅顯示OD
    how (str) : 填入'sum'或'mean'
    span (int) : 級距 (default 設為100)

    OthersObject:
    countdf (dataframe):統計出來的OD表 (尚未轉成geodataframe)
    countgdf(geodataframe) : OD帶寬 的 geodataframe 
    '''

    # 1. 不合併雙向OD
    if combine == False : 
        if how != 'countd':
            countdf = df.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : how})
        elif how == 'countd':
            countdf = df.groupby([o_col, d_col]).agg({o_x_col:'mean', o_y_col:'mean',  d_x_col:'mean', d_y_col:'mean', count_col : 'sum', date_col:'unique'})
            countdf[count_col] = countdf[count_col] / countdf[date_col]
            countdf = countdf.drop(columns = date_col)
    else :
        if how != 'countd':
            


    return countdf

